In [ ]:
# Create Sagemaker execution role
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.cnn")).upload_file(
    train_dir + "/train.cnn"
)

# This is not required as these values are the defaults:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Modify this based on your script name !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
entry_point = "cnnScript.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.large"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "script",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "image-classification-model",
}

estimator = SKLearn(**estimator_parameters)

inputs = {
    "train": pickle_train_s3_uri
}

# starting the training job
estimator.fit(inputs)

sklearn_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.large',
                                     endpoint_name='imageclassification-endpoint')

sklearn_predictor.predict([[0],[1],[2],[3]])

sklearn_predictor.delete_endpoint(True)